In [1]:
using Random, Distributions, LinearAlgebra, Statistics, DelimitedFiles

In [2]:
H_all = 0:20 # hidden state space
M_all = 1:3 # market condition space

1:3

In [3]:
# simulate a sequence of market conditions
function sim_environment(T,pm=0.5)
    m = fill(99, T)
    m[1] = sample(M_all,1)[1]
    for i = 2:T
        mbin = rand(Bernoulli(pm), 1)[1]
        if mbin==1
            m[i] = sample(M_all,1)[1]
        else
            m[i] = m[i-1]
        end
    end
    return m
end

sim_environment (generic function with 2 methods)

In [4]:
# outcome function
function mu_func(h,w)
    return h+0.5*w*h
end 

mu_func (generic function with 1 method)

In [5]:
# return a sequence of treatment assignments
function get_design(T,l,pw)
    k = ceil(Int,T/l)
    assignment = rand(Bernoulli(pw), k)
    switchback = repeat(assignment,inner=l)
    return switchback[1:T]
end 

get_design (generic function with 1 method)

In [6]:
# return indices of the burn-in periods
function get_burnin(b,l,T)
    burnin = zeros(0)
    k = ceil(Int,T/l)
    for i in 1:(k-1)
        append!(burnin, (l*(i-1)+1):(l*(i-1)+b))
    end
    append!(burnin, (l*(k-1)+1):min(l*(k-1)+b,T))
    return Int.(burnin)
end

get_burnin (generic function with 1 method)

In [7]:
function switchback_est(y,z,b,l)
    k = length(z)
    # divide y into blocks and calculate block means
    y_bar_f = [mean(y[(i-1)*l+b+1:i*l]) for i in 1:k] # y_bar in focal periods
    # calculate estimator
    if sum(z) == 0
        htau_dmf = - mean(y_bar_f[z.==0])
    elseif sum(1 .- z) == 0
        htau_dmf = mean(y_bar_f[z.==1])
    else
        htau_dmf = mean(y_bar_f[z.==1]) - mean(y_bar_f[z.==0])
    end
    # calculate jackknife variance estimator
    htau_dmf_j = zeros(k) # initialize
    for i in 1:k
        y_bar_f_j = y_bar_f[setdiff(1:k,i)] # take out i
        z_j = z[setdiff(1:k,i)]
        if sum(z_j) == 0
            htau_dmf_j[i] = - mean(y_bar_f_j[z_j.==0])
        elseif sum(1 .- z_j) == 0
            htau_dmf_j[i] = mean(y_bar_f_j[z_j.==1])
        else
            htau_dmf_j[i] = mean(y_bar_f_j[z_j.==1]) - mean(y_bar_f_j[z_j.==0])
        end
    end
    htau_dmf_var = (k-1)*mean((htau_dmf_j.-htau_dmf).^2)
    return htau_dmf, htau_dmf_var
end

switchback_est (generic function with 1 method)

In [8]:
function switchback_bc(y,z,b,l,nblocks=2)
    if b <= 0
        error("No burn-in periods!")
    end
    k = length(z)
    # divide y into blocks and calculate block means
    y_bar_b = [mean(y[(i-1)*l+1:(i-1)*l+b]) for i in 1:k] # y_bar in burn-in periods
    y_bar_f = [mean(y[(i-1)*l+b+1:i*l]) for i in 1:k] # y_bar in focal periods
    # calculate estimator
    if sum(z) == 0
        htau_dmf = - mean(y_bar_f[z.==0])
    elseif sum(1 .- z) == 0
        htau_dmf = mean(y_bar_f[z.==1])
    else
        htau_dmf = mean(y_bar_f[z.==1]) - mean(y_bar_f[z.==0])
    end
    zz1 = z[2:k] .* z[1:(k-1)] # z_i=z_{i-1}=1
    zz0 = (1 .- z[2:k]) .* (1 .- z[1:(k-1)]) # z_i=z_{i-1}=0
    if sum(zz1) == 0
        htau_bcb = - mean(y_bar_b[2:k][zz0.==1])
    elseif sum(zz0) == 0
        htau_bcb = mean(y_bar_b[2:k][zz1.==1])
    else
        htau_bcb = mean(y_bar_b[2:k][zz1.==1]) - mean(y_bar_b[2:k][zz0.==1])
    end
    htau_bc = b/l*htau_bcb + (l-b)/l*htau_dmf
    # calculate jackknife variance estimator
    htau_bc_j = zeros(k-nblocks) # initialize
    for i in 1:(k-nblocks)
        y_bar_f_j = y_bar_f[setdiff(1:k,i:(i+nblocks-1))] # take out i, i+1, ..., i+nblocks-1
        y_bar_b_j = y_bar_b[setdiff(1:k,(i+1):(i+nblocks))] # take out i+1, i+2, ..., i+nblocks
        z_j = z[setdiff(1:k,i:(i+nblocks-1))]
        zz1_j = zz1[setdiff(1:length(zz1),i:(i+nblocks-1))]
        zz0_j = zz0[setdiff(1:length(zz0),i:(i+nblocks-1))]
        if sum(z_j) == 0
            htau_dmf_j = - mean(y_bar_f_j[z_j.==0])
        elseif sum(1 .- z_j) == 0
            htau_dmf_j = mean(y_bar_f_j[z_j.==1])
        else
            htau_dmf_j = mean(y_bar_f_j[z_j.==1]) - mean(y_bar_f_j[z_j.==0])
        end
        if sum(zz1_j) == 0
            htau_bcb_j = - mean(y_bar_b_j[2:length(z_j)][zz0_j.==1])
        elseif sum(zz0_j) == 0
            htau_bcb_j = mean(y_bar_b_j[2:length(z_j)][zz1_j.==1])
        else
            htau_bcb_j = mean(y_bar_b_j[2:length(z_j)][zz1_j.==1]) - mean(y_bar_b_j[2:length(z_j)][zz0_j.==1])
        end
        htau_bc_j[i] =  b/l*htau_bcb_j + (l-b)/l*htau_dmf_j
    end
    htau_bc_var = (k-nblocks-1)^2/nblocks/(k-nblocks)*mean((htau_bc_j.-htau_bc).^2)
    return htau_bc, htau_bc_var
end

switchback_bc (generic function with 2 methods)

In [9]:
# return a sequence of outcomes (without noise)
function sim_switchback(T,m,w,mixing,ph=0.7,pw=0.5)
    
    # create matrices for storage
    h = fill(99, T)
    mu = fill(99.99, T)
    
    # get initial state
    h[1] = sample(H_all,1)[1]
    mu[1] = mu_func(h[1],w[1])
    
    # transition
    for i = 2:T
        hprob = ph*w[i] + (1-ph)*(1-w[i])
        hbin = rand(Bernoulli(hprob), 1)[1]
        if hbin==1
            h[i] = min(h[i-1]+m[i], 20)
        else
            if mixing
                h[i] = H_all[1]
            else
                h[i] = max(h[i-1]-m[i], 0)
            end
        end
        mu[i] = mu_func(h[i],w[i])
    end

    return mu
end

sim_switchback (generic function with 3 methods)

In [10]:
function sim_switchback_res(T,b,l,m,mixing,nblocks=2,sigma=3,ph=0.7,pm=0.5,pw=0.5)

    # switchback assignment
    w = get_design(T,l,pw)
    
    # outcome
    mu = sim_switchback(T,m,w,mixing,ph,pw)
    y = mu .+ rand(Normal(0,sigma), T)

    # blockwise treatment
    k = floor(Int, T/l)
    z = [mean(w[((i-1)*l+1):(i*l)]) for i in 1:k]

    # crop y
    y = y[1:(k*l)]
    
    # estimator 1 - difference-in-means
    htau_dm, htau_dm_var = switchback_est(y,z,0,l)

    # estimator 2 - difference-in-means with burn-in
    htau_dmb, htau_dmb_var = switchback_est(y,z,b,l)

    # estimator 3 - bias corrected
    htau_bc, htau_bc_var = switchback_bc(y,z,b,l,nblocks)
    
    return [htau_dm,htau_dmb,htau_bc,htau_dm_var,htau_dmb_var,htau_bc_var]
end

sim_switchback_res (generic function with 6 methods)

In [11]:
# for reproducing Table 1
seed_num = 12345
Random.seed!(seed_num)

T = 20000
l = 200
b_all = [50,100,150]
B = 1000

# simulate market condition and fix it
m = sim_environment(T)

# calculate estimand values
res_tau = zeros(B)
res_tau_FATE = zeros(B,length(b_all))

for iter in 1:B
    mu1 = sim_switchback(T, m, fill(1, T), false)
    mu0 = sim_switchback(T, m, fill(0, T), false)
    res_tau[iter] = mean(mu1) - mean(mu0)
    for b_index in 1:length(b_all)
        burnin = get_burnin(b_all[b_index],l,T)
        res_tau_FATE[iter,b_index] = mean(mu1[setdiff(1:T, burnin)]) - mean(mu0[setdiff(1:T, burnin)])
    end
end

tau = mean(res_tau)
tau_FATE = mean(res_tau_FATE, dims=1)

1×3 Matrix{Float64}:
 25.6776  25.6873  25.6558

In [12]:
res_table = zeros(3,12)

for b_index in 1:length(b_all)
    Random.seed!(seed_num)
    res1 = zeros(0)
    res2 = zeros(0)
    res3 = zeros(0)
    res4 = zeros(0)
    res5 = zeros(0)
    res6 = zeros(0)
    for _ in 1:B
        result = sim_switchback_res(T,b_all[b_index],l,m,false,2)
        append!(res1,result[1])
        append!(res2,result[2])
        append!(res3,result[3])
        append!(res4,result[4])
        append!(res5,result[5])
        append!(res6,result[6])
    end
    res_table[b_index,1] = abs(mean(res1.-tau)) # bias
    res_table[b_index,2] = abs(mean(res2.-tau_FATE[b_index]))
    res_table[b_index,3] = abs(mean(res3.-tau))
    res_table[b_index,4] = sqrt(var(res1)) # standard error (truth)
    res_table[b_index,5] = sqrt(var(res2))
    res_table[b_index,6] = sqrt(var(res3))
    res_table[b_index,7] = mean(sqrt.(res4)) # standard error (jackknife)
    res_table[b_index,8] = mean(sqrt.(res5))
    res_table[b_index,9] = mean(sqrt.(res6))
    res_table[b_index,10] = mean((tau .<= res1 .+ 1.96 .* sqrt.(res4)) .& (tau .>= res1 .- 1.96 .* sqrt.(res4))) # coverage
    res_table[b_index,11] = mean((tau_FATE[b_index] .<= res2 .+ 1.96 .* sqrt.(res5)) .& (tau_FATE[b_index] .>= res2 .- 1.96 .* sqrt.(res5)))
    res_table[b_index,12] = mean((tau .<= res3 .+ 1.96 .* sqrt.(res6)) .& (tau .>= res3 .- 1.96 .* sqrt.(res6)))
end

In [13]:
display(res_table[:,1:3]) # bias(DM), bias(DMB), bias(BC)

3×3 Matrix{Float64}:
 1.17035  0.00728593  9.03093e-5
 1.17035  0.0054204   0.00437815
 1.17035  0.0041383   0.003003

In [14]:
display(res_table[:,4:6]) # standard error(DM), standard error(DMB), standard error(BC) --> truth

3×3 Matrix{Float64}:
 0.227555  0.198973  0.201825
 0.227555  0.238995  0.225951
 0.227555  0.3324    0.247674

In [15]:
display(res_table[:,7:9]) # standard error(DM), standard error(DMB), standard error(BC) --> estimated

3×3 Matrix{Float64}:
 0.235214  0.199031  0.196035
 0.235214  0.240897  0.216445
 0.235214  0.327142  0.236157

In [16]:
display(res_table[:,10:12]) # coverage(DM), coverage(DMB), coverage(BC)

3×3 Matrix{Float64}:
 0.0  0.944  0.933
 0.0  0.953  0.926
 0.0  0.931  0.925

In [17]:
# for reproducing Tables S2 and S3
seed_num = 12345
Random.seed!(seed_num)

T = 20000
l = 200
b_all = [50,100,150]
B = 1000

# simulate market condition and fix it
m = sim_environment(T)

# calculate estimand values
res_tau = zeros(B)
res_tau_FATE = zeros(B,length(b_all))

for iter in 1:B
    mu1 = sim_switchback(T, m, fill(1, T), true)
    mu0 = sim_switchback(T, m, fill(0, T), true)
    res_tau[iter] = mean(mu1) - mean(mu0)
    for b_index in 1:length(b_all)
        burnin = get_burnin(b_all[b_index],l,T)
        res_tau_FATE[iter,b_index] = mean(mu1[setdiff(1:T, burnin)]) - mean(mu0[setdiff(1:T, burnin)])
    end
end

tau = mean(res_tau)
tau_FATE = mean(res_tau_FATE, dims=1)

1×3 Matrix{Float64}:
 5.86176  5.84831  5.88713

In [18]:
res_table = zeros(3,12)

for b_index in 1:length(b_all)
    Random.seed!(seed_num)
    res1 = zeros(0)
    res2 = zeros(0)
    res3 = zeros(0)
    res4 = zeros(0)
    res5 = zeros(0)
    res6 = zeros(0)
    for _ in 1:B
        result = sim_switchback_res(T,b_all[b_index],l,m,true,2)
        append!(res1,result[1])
        append!(res2,result[2])
        append!(res3,result[3])
        append!(res4,result[4])
        append!(res5,result[5])
        append!(res6,result[6])
    end
    res_table[b_index,1] = abs(mean(res1.-tau)) # bias
    res_table[b_index,2] = abs(mean(res2.-tau_FATE[b_index]))
    res_table[b_index,3] = abs(mean(res3.-tau))
    res_table[b_index,4] = sqrt(var(res1)) # standard error (truth)
    res_table[b_index,5] = sqrt(var(res2))
    res_table[b_index,6] = sqrt(var(res3))
    res_table[b_index,7] = mean(sqrt.(res4)) # standard error (jackknife)
    res_table[b_index,8] = mean(sqrt.(res5))
    res_table[b_index,9] = mean(sqrt.(res6))
    res_table[b_index,10] = mean((tau .<= res1 .+ 1.96 .* sqrt.(res4)) .& (tau .>= res1 .- 1.96 .* sqrt.(res4))) # coverage
    res_table[b_index,11] = mean((tau_FATE[b_index] .<= res2 .+ 1.96 .* sqrt.(res5)) .& (tau_FATE[b_index] .>= res2 .- 1.96 .* sqrt.(res5)))
    res_table[b_index,12] = mean((tau .<= res3 .+ 1.96 .* sqrt.(res6)) .& (tau .>= res3 .- 1.96 .* sqrt.(res6)))
end

In [19]:
display(res_table[:,1:3]) # bias(DM), bias(DMB), bias(BC)

3×3 Matrix{Float64}:
 0.0303807  0.00107553   0.00263518
 0.0303807  0.000911573  0.00322508
 0.0303807  0.0125001    0.00533371

In [20]:
display(res_table[:,4:6]) # standard error(DM), standard error(DMB), standard error(BC) --> truth

3×3 Matrix{Float64}:
 0.175284  0.199508  0.201946
 0.175284  0.244705  0.221643
 0.175284  0.344737  0.241651

In [21]:
display(res_table[:,7:9]) # standard error(DM), standard error(DMB), standard error(BC) --> estimated

3×3 Matrix{Float64}:
 0.176166  0.203094  0.198346
 0.176166  0.24844   0.219519
 0.176166  0.346572  0.239739

In [22]:
display(res_table[:,10:12]) # coverage(DM), coverage(DMB), coverage(BC)

3×3 Matrix{Float64}:
 0.95  0.956  0.946
 0.95  0.951  0.941
 0.95  0.955  0.935

In [20]:
# for reproducing the variances ratio plot 
# using mixing=false gives the original case, 
#    while using mixing=true gives the mixing case
seed_num = 12345
Random.seed!(seed_num)
res1 = zeros(0)
res2 = zeros(0)
res3 = zeros(0)
res4 = zeros(0)
res5 = zeros(0)
res6 = zeros(0)
for _ in 1:B
    result = sim_switchback_res(T,b_all[2],l,m,false,2)
    append!(res1,result[1])
    append!(res2,result[2])
    append!(res3,result[3])
    append!(res4,result[4])
    append!(res5,result[5])
    append!(res6,result[6])
end

In [21]:
var_mat = hcat(res1, res2, res3, res4, res5, res6)

# Write the transposed matrix to a CSV file
writedlm("sim_toy_var_hard.csv", var_mat, ',')